<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab/Sweeps_with_Pytorch_Lightning_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q pytorch-lightning wandb

We make sure we're logged into W&B so that our experiments can be associated with our account.

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: tim-w. Use `wandb login --relogin` to force relogin


True

In [17]:
accuracy?

In [23]:
from torch.optim import SGD
from torch.optim import Adagrad
import torch
from torch.nn import Linear, CrossEntropyLoss, functional as F
from torch.optim import Adam
from torchmetrics.functional import accuracy
from pytorch_lightning import LightningModule
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import os
import glob
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))])

dataset = MNIST(root="./MNIST", download=True, transform=transform)
training_set, validation_set = random_split(dataset, [55000, 5000])

class MNIST_LitModule(LightningModule):

    def __init__(self, n_classes=10, n_layer_1=128, n_layer_2=256, lr=1e-3, optim = "Adam"):
        '''method used to define our model parameters'''
        super().__init__()

        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = Linear(28 * 28, n_layer_1)
        self.layer_2 = Linear(n_layer_1, n_layer_2)
        self.layer_3 = Linear(n_layer_2, n_classes)

        # loss
        self.loss = CrossEntropyLoss()

        # optimizer parameters
        self.lr = lr

        # save hyper-parameters to self.hparams (auto-logged by W&B)
        self.save_hyperparameters()

        self.optim = optim

    def forward(self, x):
        '''method used for inference input -> output'''

        batch_size, channels, width, height = x.size()

        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)

        # let's do 3 x (linear + relu)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)

        return x

    def training_step(self, batch, batch_idx):
        '''needs to return a loss from a single batch'''
        _, loss, acc = self._get_preds_loss_accuracy(batch)

        # Log loss and metric
        self.log('train_loss', loss)
        self.log('train_accuracy', acc)

        return loss

    def validation_step(self, batch, batch_idx):
        '''used for logging metrics'''
        preds, loss, acc = self._get_preds_loss_accuracy(batch)

        # Log loss and metric
        self.log('val_loss', loss)
        self.log('val_accuracy', acc)

        # Let's return preds to use it in a custom callback
        return preds

    def test_step(self, batch, batch_idx):
        '''used for logging metrics'''
        _, loss, acc = self._get_preds_loss_accuracy(batch)

        # Log loss and metric
        self.log('test_loss', loss)
        self.log('test_accuracy', acc)
    
    def configure_optimizers(self):
        '''defines model optimizer'''
        if self.optim == "Adam":
          return Adam(self.parameters(), lr=self.lr)
        elif self.optim == "SGD":
          return SGD(self.parameters(), lr = self.lr)
        elif self.optim == "Adagrad":
          return Adagrad(self.parameters(), lr = self.lr)
    
    def _get_preds_loss_accuracy(self, batch):
        '''convenience function since train/valid/test steps are similar'''
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        loss = self.loss(logits, y)
        acc = accuracy(preds=preds, target=y, task="multiclass", num_classes=10)
        return preds, loss, acc

## ⚙️ Using WandbLogger to log Images, Text and More
Pytorch Lightning is extensible through its callback system. We can create a custom callback to automatically log sample predictions during validation. `WandbLogger` provides convenient media logging functions:
* `WandbLogger.log_text` for text data
* `WandbLogger.log_image` for images
* `WandbLogger.log_table` for [W&B Tables](https://docs.wandb.ai/guides/data-vis).

An alternate to `self.log` in the Model class is directly using `wandb.log({dict})` or `trainer.logger.experiment.log({dict})`

In this case we log the first 20 images in the first batch of the validation dataset along with the predicted and ground truth labels.

## 🏋️‍ Set up sweep

In [24]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'val_loss',  ## matches what i write via SummaryWriter
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'learning_rate':{
          'values': [0.05,0.025,0.01,0.005,0.001]
      }, 
      'batch_size': { 
          'values': [128, 256]
      }
  }
}


def sweep_train(config_defaults = dict(learning_rate=0.01, batch_size = 128)): 

  config_standard = {
          "num_workers": os.cpu_count(),  # try 0, 1, and 2
          "pin_memory": True,  # try False and True
          "precision": 32,  # try 16 and 32
          "optimizer": "Adam", 
          }
  
  config = {**config_defaults, **config_standard}
  wandb_logger = WandbLogger(log_model='all', save_code=True, config=config)
  run = wandb.run 
  run.mark_preempting()

  class LogPredictionsCallback(Callback):
      
      def on_validation_batch_end(
          self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
          """Called when the validation batch ends."""
  
          # `outputs` comes from `LightningModule.validation_step`
          # which corresponds to our model predictions in this case
          
          # Let's log 20 sample image predictions from first batch
          if batch_idx == 0:
              n = 20
              x, y = batch
              images = [img for img in x[:n]]
              captions = [f'Ground Truth: {y_i} - Prediction: {y_pred}' for y_i, y_pred in zip(y[:n], outputs[:n])]
              
              # Option 1: log images with `WandbLogger.log_image`
              wandb_logger.log_image(key='sample_images', images=images, caption=captions)

              # Option 2: log predictions as a Table
              columns = ['image', 'ground truth', 'prediction']
              data = [[wandb.Image(x_i), y_i, y_pred] for x_i, y_i, y_pred in list(zip(x[:n], y[:n], outputs[:n]))]
              wandb_logger.log_table(key='sample_table', columns=columns, data=data)  

  log_predictions_callback = LogPredictionsCallback()
  checkpoint_callback = ModelCheckpoint(monitor='val_accuracy', mode='max')

  training_loader = DataLoader(training_set, batch_size=wandb.config.batch_size, shuffle=True, pin_memory=True)
  validation_loader = DataLoader(validation_set, batch_size=64, pin_memory=True)
  ## Using a raw DataLoader, rather than LightningDataModule, for greater transparency

  # Set up model
  model = MNIST_LitModule(n_layer_1=128, n_layer_2=128, optim = wandb.config.optimizer, lr = wandb.config.learning_rate)
  wandb.watch(model)


  trainer = Trainer(gpus=1, max_epochs=5,logger=wandb_logger,
                            callbacks=[
                                      log_predictions_callback
                                      ], 
                        precision=32)
  trainer.fit(model, training_loader, validation_loader)
  wandb.finish()

In [25]:
sweep_id = wandb.sweep(sweep_config, project="ptl-sweeps-example-v2")

Create sweep with ID: w35g2jx9
Sweep URL: https://wandb.ai/tim-w/ptl-sweeps-example-v2/sweeps/w35g2jx9


In [13]:
wandb_agent = wandb.agent(sweep_id, function=sweep_train, count = 3)

wandb: Agent Starting Run: fbdo5vuh with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.05
Run fbdo5vuh errored: Error('You must call wandb.init() before wandb.config.batch_size')
wandb: ERROR Run fbdo5vuh errored: Error('You must call wandb.init() before wandb.config.batch_size')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: xpvun7we with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.025
Run xpvun7we errored: Error('You must call wandb.init() before wandb.config.batch_size')
wandb: ERROR Run xpvun7we errored: Error('You must call wandb.init() before wandb.config.batch_size')
wandb: Agent Starting Run: htcio4b4 with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.01
Run htcio4b4 errored: Error('You must call wandb.init() before wandb.config.batch_size')
wandb: ERROR Run htcio4b4 errored: Error('You must call wandb.init() before wandb.config.batch_size')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wa